In [8]:
import redis
from datetime import timedelta
#ss -nltp to check localhost ports
r = redis.Redis(host='localhost', port=6379, db=0)

In [19]:
# Leaky bucket rate limiting algorithm: https://dev.to/astagi/rate-limiting-using-python-and-redis-58gk
def request_is_limited(r: redis.Redis, key: str, limit: int, period: timedelta):
    period_in_seconds = int(period.total_seconds())
    t = r.time()[0]
    separation = round(period_in_seconds / limit)
    r.setnx(key, 0)
    tat = max(int(r.get(key)), t)
    if tat - t <= period_in_seconds - separation:
        new_tat = max(tat, t) + separation
        r.set(key, new_tat)
        return False
    return True

r = redis.Redis(host='127.0.0.1', port=6379, db=0)
requests = 10

for i in range(requests):
    if request_is_limited(r, 'admin', 10, timedelta(minutes=1)):
        print ('🛑 Request is limited')
    else:
        print ('✅ Request is allowed')

✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
✅ Request is allowed
